In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/selected_train_features_with_plof_wo_cluster.csv")
test = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/selected_test_features_with_plof_wo_cluster.csv")

In [ ]:
X_train = train.drop(["plof_scores","label", "loss"], axis = 1)
X_test = test.drop(["plof_scores","label", "loss"], axis = 1)
y_train = train["label"]
y_test = test["label"]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(
    n_estimators=1200,
    max_depth=20,
    min_samples_leaf=1,
    random_state=42
)

# from sklearn.tree import DecisionTreeClassifier

# # Create a Decision Tree Classifier with custom parameters
# rfc = DecisionTreeClassifier()

In [ ]:
rfc.fit(X_train, y_train)

DecisionTreeClassifier()

In [ ]:
# Calculate the baseline accuracy
baseline_accuracy = rfc.score(X_test, y_test)


In [ ]:
def importance(X_train, y_train, X_test, y_test, per):
  feat_importances = {}
  for feat, feature_importance in zip(X_test.columns, rfc.feature_importances_):
    X_test_0 = X_test.copy()
    X_test_0[feat] = 0
    accuracy_0 = rfc.score(X_test_0, y_test)

    decreased_acc = baseline_accuracy - accuracy_0

    feat_importances[feat] = decreased_acc

  num_feat = int(len(X_test.columns) * per)
  # Sort the feature importances in ascending order
  sorted_importances = sorted(feat_importances.items(), key=lambda x: x[1])

  selected_features = [feature for feature, _ in sorted_importances[:num_feat]]

  tr = X_train[selected_features].copy()
  ts = X_test[selected_features].copy()
  train = pd.concat([tr, y_train], axis = 1)
  test = pd.concat([ts, y_test], axis = 1)
  tm = per*100
  train_path = f"/content/drive/MyDrive/Filters_colorectal/MDA/train_{tm}_mda_dt_selected.csv"
  test_path = f"/content/drive/MyDrive/Filters_colorectal/MDA/test_{tm}_mda_dt_selected.csv"
  train.to_csv(train_path, index=False)
  test.to_csv(test_path, index=False)



In [ ]:
ls = [0.2, 0.4 , 0.5 , 0.6 , 0.8]
for per in ls:
  importance(X_train, y_train, X_test, y_test, per)